# Transform Data from formatted upload to guest matrix

## Idea

  - Upload Data (~~Excel File~~ or CSV?) into Data frame guestlist
  - ~~delete empty rows (if needed~~
  - create new Dataframe "guest_df" w/ len(guestlist) as rows and columns
  - indices of guest_df = indices of guestlist
  - use dictionary for relationship values (e.g. family = 200)
  - bride/groom + cluster become one attribute
  - use loc method to look up +1
  - first and surname become one string for lookup
  - write values into guest_df if i == j -> 0; for cluster of i find all indices with the same index (loc method) -> row i fill columns j with value
  - repeat for all rows
  
  - partner/+1: extra lookup (loc method) 
  
## Open Issues

  - ~~implement partner value~~
  - implement +1 functionality (append list if +1 is given in the attending with column): problem for lookup (how to name +1 correctly)
    - idea: if attending w/ in ["+1", "+ 1", "plus one"]: partner = +1 of "fullname", append list +1 of "fullname" -> partner "fullname"

  
## Further Improvements

  - functionality for multiple values in cluster
  - derive weight from frequency of clusterIDs -> assumption smaller groups should always sit together
  - map bzw. mapapply für manipulation der Daten anschauen bzgl.

In [168]:
import pandas as pd
import numpy as np

In [169]:
# Dictionary for relationship values
weight = {"bride family":200, "groom family":200, "partner":2000}

weight["bride family"]

200

In [170]:
guestlist = pd.read_csv("../../data/test_guestlist.csv", sep = ";")
dim = len(guestlist)

guestlist["closeness"] = guestlist.apply(lambda x: " ".join(x[["Bride/Groom", "Cluster"]]), axis=1)
guestlist["full name"] = guestlist.apply(lambda x: " ".join(x[[0, 1]]), axis=1)

plusone = ["+1", "+ 1", "plus one"]

guestlist

,First name,surname,Bride/Groom,Cluster,attending w/,closeness,full name
0,one,one,bride,family,two two,bride family,one one
1,two,two,bride,family,one one,bride family,two two
2,three,three,groom,family,+1,groom family,three three
3,four,four,groom,family,NaN,groom family,four four


In [171]:
guestlist = pd.read_csv("../../data/test_guestlist.csv", sep = ";")
dim = len(guestlist)

guestlist["closeness"] = guestlist.apply(lambda x: " ".join(x[["Bride/Groom", "Cluster"]]), axis=1)
guestlist["full name"] = guestlist.apply(lambda x: " ".join(x[[0, 1]]), axis=1)

# variants how a plus one can be entered without giving a name
plusone = ["+1", "+ 1", "plus one"]

i = 0

for i in range(0, dim):
    if guestlist.loc[i, "attending w/"] in plusone:
        guestlist.loc[dim] = ["+1 of ", guestlist.loc[i, "full name"], np.nan, np.nan, guestlist.loc[i, "full name"], np.nan , "+1 of " + guestlist.loc[i, "full name"]]
        guestlist.loc[i, "attending w/"] = "+1 of " + guestlist.loc[i, "full name"]
        dim += 1

guestlist

,First name,surname,Bride/Groom,Cluster,attending w/,closeness,full name
0,one,one,bride,family,two two,bride family,one one
1,two,two,bride,family,one one,bride family,two two
2,three,three,groom,family,+1 of three three,groom family,three three
3,four,four,groom,family,NaN,groom family,four four
4,+1 of,three three,NaN,NaN,three three,NaN,+1 of three three


In [172]:
# create array with size len(guestlist) x len(guestlist) filled w/ 0s to initialise DF

guest_df = pd.DataFrame(np.random.randint(low=0, high=1, size=(dim, dim)))

guest_df

,0,1,2,3,4
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0


In [163]:
dim

5

In [164]:
guestlist["closeness"] = guestlist.apply(lambda x: " ".join(x[["Bride/Groom", "Cluster"]]), axis=1)
guestlist["full name"] = guestlist.apply(lambda x: " ".join(x[[0, 1]]), axis=1)

TypeError: ('sequence item 0: expected str instance, float found', 'occurred at index 4')

In [173]:
guestlist

,First name,surname,Bride/Groom,Cluster,attending w/,closeness,full name
0,one,one,bride,family,two two,bride family,one one
1,two,two,bride,family,one one,bride family,two two
2,three,three,groom,family,+1 of three three,groom family,three three
3,four,four,groom,family,NaN,groom family,four four
4,+1 of,three three,NaN,NaN,three three,NaN,+1 of three three


## Idea Loop Closeness Values

  - go through guestlist row by row (eg start with index 0)
  - look up closeness in that row (for i = 0 -> bride family)
  - return indices for all rows with that value (array for bride family -> 0, 1)
  
  - write values into data frame row by row:
  - rows i, columns j: 
  - j takes all values in array, if i == j -> 0 else value from dictionary (row 0: j == 0 -> 0 ; j == 1 -> 200)
  
  repeat for all i
  


In [166]:
clusterID = guestlist.loc[4, "closeness"]

In [174]:
# ERROR because NaN is not defined in dictionary

i = 0
# go through all rows
for i in range(0, dim):
   
    # look up cluster ID (equals entry in column "closeness" of that row)
    clusterID = guestlist.loc[i, "closeness"]
    partnerID = guestlist.loc[i, "attending w/"]
    
    # return an array with all row indices with the same clusterID
    congruent = np.array(np.where(guestlist["closeness"] == clusterID))
    partner = np.where(guestlist["full name"] == partnerID)
    
    # write values into guest_df: for each row write weight for clusterID
    # if i == j don't overwrite 0 
    j = 0
    for j in range (0, dim):
        if j in congruent and i != j:
            guest_df.loc[i,j] += weight[clusterID]
        
        if j in partner:
            guest_df.loc[i,j] += weight["partner"]
            
guest_df   



,0,1,2,3,4
0,0,2200,0,0,0
1,2200,0,0,0,0
2,0,0,0,200,2000
3,0,0,200,0,0
4,0,0,2000,0,0
